In [ ]:
#two body problem example

using Pkg
Pkg.activate(".")
using LinearAlgebra
using ForwardDiff
using Plots 
using TSSOS 
using DynamicPolynomials
using DifferentialEquations  

In [ ]:
#gravitational parameter for the Earth 
μ = 3.986004418e5 #km3/s2

In [ ]:
#iss initial conditions
r0 = [6791.0; 0; 0] #km
v0 = [0; cosd(51.5)*7.66; sind(51.5)*7.66] #km/s
x_initial = [r0; v0]

In [ ]:
#one iss period is around 90-93 minutes
#revs = 5

#working for 1/4 rev...
#suboptimality gap of 21% 
revs = 0.25

#finding the period
r_initial = norm(r0)

#finding the period for 1 rev
iss_period = 2*pi*sqrt(r_initial^3/μ)

period = revs*iss_period 

In [ ]:
#amount of knot points per orbit 
knot_pts = 10

In [ ]:
#total number of knot points
#N=knot_pts*revs

#when revs is less than 1
N = knot_pts

In [ ]:
#define a timestep
h = period/(N-1) 

In [ ]:
function two_body_dynamics(x)

    q = x[1:3]
    v = x[4:6]

    a = zeros(eltype(x), size(x))

    r = norm(q)
    
    a[1:3] = v

    a[4:6] = (-μ/r^3)*q

    return a

end

In [ ]:
function rk4_integrator(x)


    f1 = two_body_dynamics(x)
    f2 = two_body_dynamics(x+0.5*h*f1)
    f3 = two_body_dynamics(x+0.5*h*f2)
    f4 = two_body_dynamics(x+h*f3)

    xnext = x + (h/6.0)*(f1+2*f2+2*f3+f4)

    return xnext 

end

In [ ]:
#integrate with symplectic euler to see what happens 

In [ ]:
#semi implicit euler 

#input is state at timestep k 
#x = [qk, vk]

function symplectic_euler(x)

    #the q and the v are already scaled 

    #qk
    q = x[1:3]

    #vk
    v = x[4:6]

    #next timestep
    xnext = zeros(6)

    ak = (-μ/(norm(q)^3))*q

    #explicit euler on the velocities
    #vk1
    xnext[4:6] = v + h*ak

    #implicit euler on the positions
    #qk1
    xnext[1:3] = q + h*xnext[4:6]

    #xnext = [xk1, vk1] (state at timestep k+1)
    return xnext 

end

In [ ]:
#integrate the orbit with symplectic euler to see if the orbit closes

xtraj = zeros(6, N)

xtraj[:,1] = x_initial

In [ ]:
for i=1:N-1

    xtraj[:,i+1] = symplectic_euler(xtraj[:,i])

end

In [ ]:
xtraj 

In [ ]:
plot(xtraj[1,:], xtraj[2,:], xtraj[3,:])

In [ ]:
#not completely periodic. off in the y and the z be 10's of km

In [ ]:
#integrate with rk4 to see how much better it is compared to symplectic euler
x_traj_rk4 = zeros(6, N)

In [ ]:
x_traj_rk4[:,1] = x_initial 

In [ ]:
for i = 1:N-1

    x_traj_rk4[:,i+1] = rk4_integrator(x_traj_rk4[:,i])

end

In [ ]:
plot(x_traj_rk4[1,:], x_traj_rk4[2,:], x_traj_rk4[3,:])

In [ ]:
#difference between the symplectic euler and rk4 final positions after 5 rev sim
x_traj_rk4[:,end] - xtraj[:,end] 

In [ ]:
function two_body_dynamics_form!(du, u, p, t)

    du[1:6] = two_body_dynamics(u[1:6])

end

In [ ]:
#rk78 integrator using DifferentialEquations.jl
function two_body_dynamics_integrate(x_0, period)

    tspan = (0.0, period)
    prob = ODEProblem(two_body_dynamics_form!, x_0, tspan)
    sol = solve(prob, TsitPap8(), abstol=1e-12, reltol=1e-12)


    return sol 
end

In [ ]:
function get_state(solution)
    
    N = size(solution.u)[1]

    all_states = zeros(6, N)

    for i=1:N
        all_states[:,i] = solution.u[i][1:6]
    end
    
    #all states and all stm are functions of t
    #solution.t is the time
    return all_states
end

In [ ]:
period  

In [ ]:
#use rk78 to integrate the initial condition to 5 revs and check the trajectory 
final_state_sol = two_body_dynamics_integrate(x_initial, period)

In [ ]:
all_states = get_state(final_state_sol)

In [ ]:
plot(all_states[1,:], all_states[2,:], all_states[3,:]) 

In [ ]:
size(all_states)[2]

In [ ]:
#find a good position and velocity scale 

position_normz = zeros(size(all_states)[2])

velocity_normz = zeros(size(all_states)[2])

for i=1:size(all_states)[2]

    position_normz[i] = norm(all_states[1:3, i])

    velocity_normz[i] = norm(all_states[4:6, i])

end

In [ ]:
position_scale = maximum(position_normz) 

#try setting the timescale to scale the period to 1 
#works for the ipopt example 
time_scale = period  

#scale the maximum velocity to 1
#time_scale = position_scale/maximum(velocity_normz)

velocity_scale = position_scale/time_scale

acceleration_scale = position_scale/time_scale^2

μ_scaled = μ/(position_scale^3/time_scale^2)

h_scaled = h/time_scale

In [ ]:
time_scale 

In [ ]:
position_scale 

In [ ]:
#difference between ground truth (rk78) and the two integrators (rk4 and semi implicit euler)

diff_semi_euler = all_states[:,end] - xtraj[:,end]

In [ ]:
diff_rk4 = all_states[:,end] - x_traj_rk4[:,end]

In [ ]:
#going to try the polynomial optimization 

@polyvar q[1:3, 1:N]
@polyvar v[1:3, 1:N]

@polyvar a[1:3, 1:N]

@polyvar r[1:N]

@polyvar u[1:3,1:N-1]

In [ ]:

Qq = Matrix(1.0*I, 3,3)

Qv = Matrix(1.0*I, 3,3)

In [ ]:
x_initial 

In [ ]:
#set in kiloNewtons (kN)
umax = 20e-9

#in scaled units 
umax_bound = umax/acceleration_scale

In [ ]:
#found visually from the unconstrained control case
#this is in the scaled units 

#with the acceleration scale, this corresponds to a maximum of 0.4 N 
#this works 
#u_max_bound = 0.05



#testing 
#doesn't work. maybe not bounding the control space enough 
#u_max_bound = 1/acceleration_scale

In [ ]:
#cost function 

f=0

#decreasae the semi major axis
for i=1:N

    f+= 1.0*r[i]

end


#add a cost on controls? (maybe), but it would avoid the bang-bang solution we want 
#introduce a cost or constraint or both

#for i=1:N-1

    #kinda works

    #    f += u[:,i]'*u[:,i]
#end


In [ ]:
#initialize equality and inequality constraints 
eq = []
ineq = []

In [ ]:
x_initial 

In [ ]:
x_initial_scaled = [x_initial[1:3]/position_scale; x_initial[4:6]/velocity_scale]

In [ ]:
#set the initial condition constraints
append!(eq, [q[1,1] - x_initial_scaled[1]])
append!(eq, [q[2,1] - x_initial_scaled[2]])
append!(eq, [q[3,1] - x_initial_scaled[3]])

append!(eq, [v[1,1] - x_initial_scaled[4]])
append!(eq, [v[2,1] - x_initial_scaled[5]])
append!(eq, [v[3,1] - x_initial_scaled[6]])

In [ ]:
#scaling all the units

In [ ]:
#set the dynamics constraint between each timestep 

for i=1:N

    #acceleration constraint
    append!(eq, a[:,i]*r[i]^3 + μ_scaled*q[:,i])

    #slack r constraint 
    append!(eq, [r[i]^2 - q[:,i]'*q[:,i]])

end

In [ ]:
#set a constraint on the r's
for i=1:N

    #lower bound on r that we want to reach 
    append!(ineq, [r[i]-0.75])

    #r is always positive (redundant constraint)
    append!(ineq, [r[i]])


end

In [ ]:
#two types of control constraints. bound constraint 
# and norm constraint (socp constraint). 
#not sure if it can be handeled without squaring it and making it a polynomial 

#bound constraint 
for i=1:N-1

    #append!(ineq, norm(u_max_bound - u[1,i], 1))
    #maybe it can't handle the norm?
    #append!(ineq, norm(u[1,i] - u_max_bound, 1))


    #bound constraints
    #order 1 polynomial 
    #two inequalities to represent the absolute value 
    #g(x) >= 0
    append!(ineq, [umax_bound - u[1,i]])
    append!(ineq, [u[1,i] + umax_bound])

    append!(ineq, [umax_bound - u[2,i]])
    append!(ineq, [u[2,i] + umax_bound])

    append!(ineq, [umax_bound - u[3,i]])
    append!(ineq, [u[3,i] + umax_bound])


end

In [ ]:
#check all the inequality constraints
ineq 

In [ ]:
#add a control constraint. square the socp constraint to make it a polynomial

#leaving out for now..
# for i=1:N-1

#     append!(ineq, umax_scaled^2 - u[:,i]'*u[:,i])

# end

In [ ]:
#check equality constraints
eq 

In [ ]:
#assuming mass is 1

In [ ]:
for i=1:N-1

    #velocity dynamics constraint
    #added a control  
    append!(eq, v[:,i+1] - (v[:,i] + h_scaled*(a[:,i] + u[:,i])))

    #position dynamics constraint 
    append!(eq, q[:,i+1] - (q[:,i] + h_scaled*v[:,i+1]))

end

In [ ]:
#add the inequality constraint
pop = append!([f], ineq)

#add the equality constraint
pop = append!(pop, eq)

In [ ]:
#relaxation order
d = 2

In [ ]:
#concatenate all the variables together 
var = [vec(q); vec(v); vec(a); r; vec(u)]

In [ ]:
#solve 
#TS defines term sparsity 
#without the control bound constraint it takes 18 seconds for 5 revs!
#the local solver is unable to refine the solution for 5 revs because the solution to the sdp is a 
#bad initial guess to the nlp solver. it doesn't satisfy the constraints...

#kinda working with some supoptimality gap for 1/4 rev 
opt, sol, data = cs_tssos_first(pop, var, d, numeq=length(eq), TS="MD", solution=true)

In [ ]:
#the data.moment extracts the moment matrix and it is a vector of 
#matrices that construct the block diagonals of the moment matrix

In [ ]:
all_condition_numbers = zeros(size(data.moment)[1]) 

for i=1:size(data.moment)[1]

    all_condition_numbers[i] = cond(data.moment[i])
    
end

In [ ]:
findmax(all_condition_numbers) 

In [ ]:
#need to reshape this 
x_traj_pop = sol[1: size(vec(q))[1]]

x_traj_pop = reshape(x_traj_pop, 3, N)

In [ ]:
norm(x_traj_pop[:,1]) 

In [ ]:
v_traj_pop = sol[size(vec(q))[1]+1: size(vec(q))[1] + size(vec(v))[1]]

v_traj_pop = reshape(v_traj_pop, 3, N)

In [ ]:
a_traj_pop = sol[size(vec(q))[1]+size(vec(v))[1]+1: size(vec(q))[1] + size(vec(v))[1] + size(vec(a))[1]]

a_traj_pop = reshape(a_traj_pop, 3, N)

In [ ]:
r_traj_pop = sol[size(vec(q))[1]+size(vec(v))[1]+size(vec(a))[1]+1: size(vec(q))[1]+size(vec(v))[1]+size(vec(a))[1] + size(vec(r))[1]]

In [ ]:
u_traj_pop = sol[size(vec(q))[1]+size(vec(v))[1]+size(vec(a))[1]+size(vec(r))[1]+1: end]

u_traj_pop = reshape(u_traj_pop, 3, N-1)

In [ ]:
plot(u_traj_pop[1,:]*acceleration_scale*1e3, title="x control", ylabel="Control (N)", xlabel="Timestep") 

In [ ]:
plot(u_traj_pop[2,:]*acceleration_scale*1e3, title="y control", ylabel="Control (N)", xlabel="Timestep") 

In [ ]:
plot(u_traj_pop[3,:]*acceleration_scale*1e3, title="z control", ylabel="Control (N)", xlabel="Timestep") 

In [ ]:
plot(x_traj_pop[1,:], x_traj_pop[2,:], x_traj_pop[3,:])

#plot!(xtraj[1,:]/position_scale, xtraj[2,:]/position_scale, xtraj[3,:]/position_scale)

In [ ]:
difference = x_traj_pop - xtraj[1:3, :]/position_scale 

In [ ]:
#r constraint not being satisfied...

In [ ]:
#check the initial condition. constraint not satisfied...
x_traj_pop[:,1] - x_initial_scaled[1:3]

In [ ]:
v_traj_pop[:,1] - x_initial_scaled[4:6]

In [ ]:
#check the constraints: 

position_constraint = zeros(3,N-1)

velocity_constraint = zeros(3,N-1)

acceleration_constraint = zeros(3,N)

r_constraint = zeros(N)

In [ ]:
x_traj_pop 

In [ ]:
for i=1:N-1

    
    velocity_constraint[:,i] = v_traj_pop[:,i+1] - (v_traj_pop[:,i] + h_scaled*(a_traj_pop[:,i] + u_traj_pop[:,i]))

    position_constraint[:,i] = x_traj_pop[:,i+1] - (x_traj_pop[:,i] + h_scaled*v_traj_pop[:,i+1])
    

end

In [ ]:
for i=1:N

    acceleration_constraint[:,i] = a_traj_pop[:,i]*r_traj_pop[i]^3 + μ_scaled*x_traj_pop[:,i]

    r_constraint[i] = r_traj_pop[i]^2 - x_traj_pop[:,i]'*x_traj_pop[:,i]

end

In [ ]:
position_constraint  

In [ ]:
plot(position_constraint')

In [ ]:
plot(velocity_constraint')

In [ ]:
plot(acceleration_constraint')

In [ ]:
#satisfies the constraints for 0.25 revs...